## Benchmark model

In this notebook a Simple Moving Average model will be created in order to have a benchmark for my DeepAR based model (to be tuned in the next notebook).

### Hyperparameters

DeepAR is the model of choice for this project.
This model expects input data to be already test-train splitted.
A big part of the model design has to be done looking close at data.
More specifically, defining these two hyperparameters about the data:
* Context length
* Prediction length

### Prediction length

This is the length of the time series future predictions in days. It will be conveniently set to 5 days (exactly a week of trading hours) because a shorter interval would be of little significance.
A longer interval could be interesting from an application point of view, but it can be challenging in terms of model performances.

### Context length

Context length can be either:
* designed on patterns or seasonality observed in the data, if any is present;
* chosen as a fixed value. This will be my choice, and it will be the same as the moving average window, in order to have a good reference metrics, applicable to both this model and the benchmark model.

To explore this second option, we will refer to what we've found during the EDA stage.

In [139]:
# declaring prediction legnth and context length
prediction_length = [10, 20, 50]
context_length = [10, 20, 50]

### Train, test and validation split

Time series will be all trimmed from `data_start` in `1.ExploratoryDataAnalysis.ipy`, according to DeepAR documentation, train time series should be set as the size of the entire time series less the prediction length.
Validation length will exactly equal to prediction length.

In [140]:
from data.data_utils import train_test_valid_split

#### IBM Stock train, test and validation split

In [141]:
#df_ibm = df_ibm.loc[data_start:].copy()

In [142]:
df_ibm_train, df_ibm_test, df_ibm_valid = train_test_valid_split(df_ibm, prediction_length=prediction_length[1])
print(len(df_ibm), len(df_ibm_train), len(df_ibm_test), len(df_ibm_valid))

14931 14891 14911 20


In [143]:
df_ibm_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
1962-01-02,1.858243,7.626667,7.713333,7.626667,7.713333,390000,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-03,1.874485,7.693333,7.693333,7.626667,7.626667,292500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-04,1.855805,7.616667,7.693333,7.613333,7.693333,262500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-05,1.819257,7.466667,7.606667,7.453333,7.606667,367500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-08,1.785148,7.326667,7.460000,7.266667,7.460000,547500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568


In [144]:
df_ibm_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
1962-01-02,1.858243,7.626667,7.713333,7.626667,7.713333,390000,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-03,1.874485,7.693333,7.693333,7.626667,7.626667,292500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-04,1.855805,7.616667,7.693333,7.613333,7.693333,262500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-05,1.819257,7.466667,7.606667,7.453333,7.606667,367500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568
1962-01-08,1.785148,7.326667,7.460000,7.266667,7.460000,547500,0.001,0.0,1.831034,1.806507,1.787152,1.885422,1.776646,1.878527,1.734487,1.854736,1.719568


In [145]:
df_ibm_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-29,135.860001,135.860001,137.070007,135.509995,135.979996,4622700,NaN,NaN,131.317001,127.949501,123.835902,137.023710,125.610292,136.997695,118.901307,133.973408,113.698396
2021-03-30,134.720001,134.720001,136.270004,134.020004,135.860001,4790400,NaN,NaN,131.965001,128.669001,123.996363,137.589873,126.340129,137.450691,119.887311,134.563587,113.429139
2021-03-31,133.259995,133.259995,134.710007,132.710007,134.539993,4945100,NaN,NaN,132.388000,129.214001,124.115189,137.656846,127.119155,137.694867,120.733135,134.964745,113.265634
2021-04-01,133.229996,133.229996,133.929993,132.270004,133.759995,4074000,NaN,NaN,132.705000,129.870000,124.212496,137.727001,127.682999,137.357293,122.382708,135.302224,113.122767
2021-04-05,135.929993,135.929993,136.690002,133.399994,133.639999,5471300,NaN,NaN,133.408000,130.525000,124.332816,138.013626,128.802374,137.704859,123.345141,135.799180,112.866452


#### Apple Inc. Stock train test split

In [146]:
#df_aapl = df_aapl.loc[data_start:].copy()

In [147]:
df_aapl_train, df_aapl_test, df_aapl_valid = train_test_valid_split(df_aapl, prediction_length=prediction_length[1])
print(len(df_aapl), len(df_aapl_train), len(df_aapl_test), len(df_aapl_valid))

10178 10138 10158 20


In [148]:
df_aapl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
1980-12-12,0.100922,0.128348,0.128906,0.128348,0.128348,469033600.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-15,0.095657,0.121652,0.122210,0.121652,0.122210,175884800.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-16,0.088636,0.112723,0.113281,0.112723,0.113281,105728000.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-17,0.090830,0.115513,0.116071,0.115513,0.115513,86441600.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-18,0.093463,0.118862,0.119420,0.118862,0.118862,73449600.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811


In [149]:
df_aapl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
1980-12-12,0.100922,0.128348,0.128906,0.128348,0.128348,469033600.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-15,0.095657,0.121652,0.122210,0.121652,0.122210,175884800.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-16,0.088636,0.112723,0.113281,0.112723,0.113281,105728000.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-17,0.090830,0.115513,0.116071,0.115513,0.115513,86441600.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811
1980-12-18,0.093463,0.118862,0.119420,0.118862,0.118862,73449600.0,0.000536,0.0,0.101975,0.108974,0.104248,0.124365,0.079585,0.132002,0.085946,0.126685,0.081811


In [150]:
df_aapl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-29,121.389999,121.389999,122.580002,120.730003,121.650002,80819200.0,NaN,NaN,122.005999,121.659999,128.347683,125.973606,118.038392,125.971542,117.348457,142.775868,113.919498
2021-03-30,119.900002,119.900002,120.400002,118.860001,120.110001,85671900.0,NaN,NaN,121.438999,121.398999,128.206677,124.700732,118.177267,125.452930,117.345068,142.827127,113.586227
2021-03-31,122.150002,122.150002,123.519997,121.150002,121.650002,118323800.0,NaN,NaN,121.177999,121.403499,128.096892,123.556867,118.799131,125.460718,117.346280,142.816832,113.376952
2021-04-01,123.000000,123.000000,124.180000,122.489998,123.660004,74957400.0,NaN,NaN,121.424999,121.546999,127.920232,124.008921,118.841077,125.617562,117.476437,142.669146,113.171318
2021-04-05,125.900002,125.900002,126.160004,123.070000,123.870003,88651200.0,NaN,NaN,122.016000,121.771000,127.704917,125.636685,118.395314,126.281432,117.260567,142.245576,113.164257


#### Amazon Stock train test split

In [151]:
#df_amzn = df_amzn.loc[data_start:].copy()

In [152]:
df_amzn_train, df_amzn_test, df_amzn_valid = train_test_valid_split(df_amzn, prediction_length=prediction_length[1])
print(len(df_amzn), len(df_amzn_train), len(df_amzn_test), len(df_amzn_valid))

6026 5986 6006 20


In [153]:
df_amzn_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
1997-05-15,1.958333,1.958333,2.500000,1.927083,2.437500,72156000.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-16,1.729167,1.729167,1.979167,1.708333,1.968750,14700000.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-19,1.708333,1.708333,1.770833,1.625000,1.760417,6106800.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-20,1.635417,1.635417,1.750000,1.635417,1.729167,5467200.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-21,1.427083,1.427083,1.645833,1.375000,1.635417,18853200.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822


In [154]:
df_amzn_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
1997-05-15,1.958333,1.958333,2.500000,1.927083,2.437500,72156000.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-16,1.729167,1.729167,1.979167,1.708333,1.968750,14700000.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-19,1.708333,1.708333,1.770833,1.625000,1.760417,6106800.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-20,1.635417,1.635417,1.750000,1.635417,1.729167,5467200.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822
1997-05-21,1.427083,1.427083,1.645833,1.375000,1.635417,18853200.0,0.0,2.0,1.597396,1.57474,1.772813,1.93311,1.261682,1.836333,1.313146,2.458803,1.086822


In [155]:
df_amzn_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-29,3075.729980,3075.729980,3091.250000,3028.449951,3055.439941,2746000.0,NaN,NaN,3084.000024,3063.738013,3176.370786,3157.227443,3010.772606,3164.265826,2963.210200,3411.929864,2940.811709
2021-03-30,3055.290039,3055.290039,3073.000000,3034.000000,3070.010010,2337600.0,NaN,NaN,3080.343018,3061.776013,3175.391587,3155.454250,3005.231786,3161.300120,2962.251907,3412.575777,2938.207397
2021-03-31,3094.080078,3094.080078,3119.330078,3062.500000,3064.060059,3093900.0,NaN,NaN,3076.178027,3066.230017,3174.857988,3141.638239,3010.717816,3162.990438,2969.469596,3412.663067,2937.052909
2021-04-01,3161.000000,3161.000000,3162.439941,3115.550049,3117.939941,2940300.0,NaN,NaN,3089.479028,3075.401514,3172.810391,3164.741096,3014.216961,3171.549083,2979.253944,3409.263633,2936.357148
2021-04-05,3226.729980,3226.729980,3235.959961,3161.239990,3173.000000,3334900.0,NaN,NaN,3104.656030,3086.715015,3171.205190,3218.319195,2990.992865,3197.819375,2975.610655,3405.015421,2937.394960


#### Alphabet Inc. Stock train test split

In [156]:
#df_googl = df_googl.loc[data_start:].copy()

In [157]:
df_googl_train, df_googl_test, df_googl_valid = train_test_valid_split(df_googl, prediction_length=prediction_length[1])
print(len(df_googl), len(df_googl_train), len(df_googl_test), len(df_googl_valid))

4200 4160 4180 20


In [158]:
df_googl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [159]:
df_googl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [160]:
df_googl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-29,2045.790039,2045.790039,2049.209961,2005.449951,2022.160034,1324600.0,NaN,NaN,2042.193970,2045.892987,2011.236191,2087.638685,1996.749255,2099.706184,1992.079790,2202.333129,1820.139254
2021-03-30,2046.469971,2046.469971,2062.669922,2036.150024,2052.979980,1365600.0,NaN,NaN,2038.451978,2044.992487,2017.613191,2073.643257,2003.260698,2098.094126,1991.890848,2190.491808,1844.734574
2021-03-31,2062.520020,2062.520020,2086.260010,2047.000000,2054.399902,2128500.0,NaN,NaN,2036.481982,2047.547986,2023.174192,2061.526685,2011.437280,2098.729389,1996.366583,2182.824467,1863.523917
2021-04-01,2129.780029,2129.780029,2135.750000,2091.429932,2092.250000,1994000.0,NaN,NaN,2047.325989,2052.340485,2028.168394,2109.546399,1985.105579,2114.849593,1989.831376,2185.147387,1871.189401
2021-04-05,2218.959961,2218.959961,2228.989990,2145.209961,2147.149902,2425500.0,NaN,NaN,2066.525989,2058.434979,2034.864592,2189.575480,1943.476497,2154.218236,1962.651723,2195.294532,1874.434652


### Test and Validation Data sets Retro-fix

It is clear that the above initialized values has introduced some kind of data leakage into test and validation data sets, from the moment that SMA computation should be stopped at the end of train time series. In order to solve this issue, I will set all the simple moving average values in the validation set and in the last part (last 20 values) of test set to the value of the last one in the training set.

#### IBM Data

In [161]:
# set moving average and bolligner bands columns values to same values in last test set record
for mds in win_szs:
    ma_column_name = "%s_ac_ma" %(str(mds))
    df_ibm_valid[ma_column_name] = df_ibm_train[ma_column_name][-1]
    bb_column_name_u = "%s_ac_bb_u" %(str(mds))
    df_ibm_valid[bb_column_name_u] = df_ibm_train[bb_column_name_u][-1]
    bb_column_name_l = "%s_ac_bb_l" %(str(mds))
    df_ibm_valid[bb_column_name_l] = df_ibm_train[bb_column_name_l][-1]

#### AAPL Validation Data

In [162]:
# set moving average and bolligner bands columns values to same values in last test set record
# set moving average and bolligner bands columns values to same values in last test set record
for mds in win_szs:
    ma_column_name = "%s_ac_ma" %(str(mds))
    df_aapl_valid[ma_column_name] = df_aapl_train[ma_column_name][-1]
    bb_column_name_u = "%s_ac_bb_u" %(str(mds))
    df_aapl_valid[bb_column_name_u] = df_aapl_train[bb_column_name_u][-1]
    bb_column_name_l = "%s_ac_bb_l" %(str(mds))
    df_aapl_valid[bb_column_name_l] = df_aapl_train[bb_column_name_l][-1]

In [163]:
df_aapl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-29,121.389999,121.389999,122.580002,120.730003,121.650002,80819200.0,NaN,NaN,127.843999,131.502835,131.838728,137.404706,118.283293,141.753261,121.252408,141.634587,122.04287
2021-03-30,119.900002,119.900002,120.400002,118.860001,120.110001,85671900.0,NaN,NaN,127.843999,131.502835,131.838728,137.404706,118.283293,141.753261,121.252408,141.634587,122.04287
2021-03-31,122.150002,122.150002,123.519997,121.150002,121.650002,118323800.0,NaN,NaN,127.843999,131.502835,131.838728,137.404706,118.283293,141.753261,121.252408,141.634587,122.04287
2021-04-01,123.000000,123.000000,124.180000,122.489998,123.660004,74957400.0,NaN,NaN,127.843999,131.502835,131.838728,137.404706,118.283293,141.753261,121.252408,141.634587,122.04287
2021-04-05,125.900002,125.900002,126.160004,123.070000,123.870003,88651200.0,NaN,NaN,127.843999,131.502835,131.838728,137.404706,118.283293,141.753261,121.252408,141.634587,122.04287


#### AMZN Validation Data

In [164]:
# set moving average and bolligner bands columns values to same values in last test set record
for mds in win_szs:
    ma_column_name = "%s_ac_ma" %(str(mds))
    df_amzn_valid[ma_column_name] = df_amzn_train[ma_column_name][-1]
    bb_column_name_u = "%s_ac_bb_u" %(str(mds))
    df_amzn_valid[bb_column_name_u] = df_amzn_train[bb_column_name_u][-1]
    bb_column_name_l = "%s_ac_bb_l" %(str(mds))
    df_amzn_valid[bb_column_name_l] = df_amzn_train[bb_column_name_l][-1]

In [165]:
df_amzn_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-29,3075.729980,3075.729980,3091.250000,3028.449951,3055.439941,2746000.0,NaN,NaN,3211.828955,3260.984961,3231.552573,3393.225455,3030.432455,3435.325008,3086.644913,3387.91579,3075.189357
2021-03-30,3055.290039,3055.290039,3073.000000,3034.000000,3070.010010,2337600.0,NaN,NaN,3211.828955,3260.984961,3231.552573,3393.225455,3030.432455,3435.325008,3086.644913,3387.91579,3075.189357
2021-03-31,3094.080078,3094.080078,3119.330078,3062.500000,3064.060059,3093900.0,NaN,NaN,3211.828955,3260.984961,3231.552573,3393.225455,3030.432455,3435.325008,3086.644913,3387.91579,3075.189357
2021-04-01,3161.000000,3161.000000,3162.439941,3115.550049,3117.939941,2940300.0,NaN,NaN,3211.828955,3260.984961,3231.552573,3393.225455,3030.432455,3435.325008,3086.644913,3387.91579,3075.189357
2021-04-05,3226.729980,3226.729980,3235.959961,3161.239990,3173.000000,3334900.0,NaN,NaN,3211.828955,3260.984961,3231.552573,3393.225455,3030.432455,3435.325008,3086.644913,3387.91579,3075.189357


#### GOOGL Validation Data

In [166]:
# set moving average and bolligner bands columns values to same values in last test set record
for mds in win_szs:
    ma_column_name = "%s_ac_ma" %(str(mds))
    df_googl_valid[ma_column_name] = df_googl_train[ma_column_name][-1]
    bb_column_name_u = "%s_ac_bb_u" %(str(mds))
    df_googl_valid[bb_column_name_u] = df_googl_train[bb_column_name_u][-1]
    bb_column_name_l = "%s_ac_bb_l" %(str(mds))
    df_googl_valid[bb_column_name_l] = df_googl_train[bb_column_name_l][-1]

In [167]:
df_googl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-29,2045.790039,2045.790039,2049.209961,2005.449951,2022.160034,1324600.0,NaN,NaN,2075.502039,2046.552502,1884.603799,2147.768484,2003.235593,2202.561915,1890.54309,2183.58537,1585.622228
2021-03-30,2046.469971,2046.469971,2062.669922,2036.150024,2052.979980,1365600.0,NaN,NaN,2075.502039,2046.552502,1884.603799,2147.768484,2003.235593,2202.561915,1890.54309,2183.58537,1585.622228
2021-03-31,2062.520020,2062.520020,2086.260010,2047.000000,2054.399902,2128500.0,NaN,NaN,2075.502039,2046.552502,1884.603799,2147.768484,2003.235593,2202.561915,1890.54309,2183.58537,1585.622228
2021-04-01,2129.780029,2129.780029,2135.750000,2091.429932,2092.250000,1994000.0,NaN,NaN,2075.502039,2046.552502,1884.603799,2147.768484,2003.235593,2202.561915,1890.54309,2183.58537,1585.622228
2021-04-05,2218.959961,2218.959961,2228.989990,2145.209961,2147.149902,2425500.0,NaN,NaN,2075.502039,2046.552502,1884.603799,2147.768484,2003.235593,2202.561915,1890.54309,2183.58537,1585.622228


In [169]:
n = 20
ma_str = str(n)+'_ac_ma'

Now I'll initialize an array of moving average values on test and validation time series, to be used in future model comparison:

In [170]:
ser_valid_ma_bmk = [df_ibm_valid[:][ma_str], df_aapl_valid[:][ma_str], df_amzn_valid[:][ma_str], df_googl_valid[:][ma_str]]

For test set, the last `prediction_length[1] = 20` days will be used for benchmark:

In [171]:
ser_test_ma_bmk = [df_ibm_valid[:][ma_str], df_aapl_valid[:][ma_str], df_amzn_valid[:][ma_str], df_googl_valid[:][ma_str]]

Fixing time index in test data:

In [172]:
ser_test_ma_bmk[0].index = df_ibm_test[-prediction_length[1]:].index
ser_test_ma_bmk[1].index = df_aapl_test[-prediction_length[1]:].index
ser_test_ma_bmk[2].index = df_amzn_test[-prediction_length[1]:].index
ser_test_ma_bmk[3].index = df_googl_test[-prediction_length[1]:].index

# Metrics computation

Computing metrics on the benchmark model will give me a good reference to evaluate the deep learning model after training.

In the cas

As can be presumed and seen from data prints, first `n` values of validation data are `NaN` so, it will be better just to exclude the first `n-1` values from the validation set to avoid misleading values to be catch in metrics evaluation. 

In [173]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

## Predicitons on test data

#### IBM stock

In [174]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [175]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [176]:
#print(ibm_ma_mse_loss)

In [177]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [178]:
ibm_ma_mae_loss = mean_absolute_error(df_ibm_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[0])

In [179]:
print(ibm_ma_mae_loss)

6.935802841186524


Root Mean Squared Error

In [180]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[0], squared=False)

In [181]:
print(ibm_ma_mse_loss)

8.137602661982527


Mean Absolute Percentage Error

In [182]:
ibm_ma_map_loss = mean_absolute_percentage_error(df_ibm_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[0])

In [183]:
print(ibm_ma_map_loss)

0.053460874159345785


R<sup>2</sup> score

In [184]:
ibm_ma_r2_score = r2_score(df_ibm_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[0])

In [185]:
print(ibm_ma_r2_score)

-2.61020840469705


In [186]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [187]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [188]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [189]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [190]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [191]:
#print(aapl_ma_mse_loss)

In [192]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [193]:
aapl_ma_mae_loss = mean_absolute_error(df_aapl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[1])

In [194]:
print(aapl_ma_mae_loss)

9.52283554077149


##### Root Mean Squared Error

In [195]:
aapl_ma_rmse_loss = mean_squared_error(df_aapl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[1], squared=False)

In [196]:
print(aapl_ma_rmse_loss)

9.842366587630263


##### Mean Absolute Percentage Error

In [197]:
aapl_ma_map_loss = mean_absolute_percentage_error(df_aapl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[1])

In [198]:
print(aapl_ma_map_loss)

0.0785150630715565


R<sup>2</sup> score

In [199]:
aapl_ma_r2_score = r2_score(df_aapl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[1])

In [200]:
print(aapl_ma_r2_score)

-14.655393088634224


In [201]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [202]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [203]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [204]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [205]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [206]:
#print(amzn_ma_mse_loss)

In [207]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [208]:
amzn_ma_mae_loss = mean_absolute_error(df_amzn_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[2])

In [209]:
print(amzn_ma_mae_loss)

193.72645263671885


Root Mean Squared Error

In [210]:
amzn_ma_rmse_loss = mean_squared_error(df_amzn_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[2], squared=False)

In [211]:
print(amzn_ma_rmse_loss)

200.62401395328592


##### Mean Absolute Percentage Error

In [212]:
amzn_ma_map_loss = mean_absolute_percentage_error(df_amzn_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[2])

In [213]:
print(amzn_ma_map_loss)

0.06346979710956135


R<sup>2</sup> score

In [214]:
amzn_ma_r2_score = r2_score(df_amzn_valid.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[2])

In [215]:
print(amzn_ma_r2_score)

-0.07217396529979303


In [216]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [217]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [218]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [219]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [220]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [221]:
#print(googl_ma_mse_loss)

In [222]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [223]:
googl_ma_mae_loss = mean_absolute_error(df_googl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[3])

In [224]:
print(googl_ma_mae_loss)

22.454000244140616


Root Mean Squared Error

In [225]:
googl_ma_rmse_loss = mean_squared_error(df_googl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[3], squared=False)

In [226]:
print(googl_ma_rmse_loss)

26.737058543014086


##### Mean Absolute Percentage Error

In [227]:
googl_ma_map_loss = mean_absolute_percentage_error(df_googl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[3])

In [228]:
print(googl_ma_map_loss)

0.010929114632073753


R<sup>2</sup> score

In [229]:
googl_ma_r2_score = r2_score(df_googl_test.iloc[-prediction_length[1]:]['Adj Close'], ser_test_ma_bmk[3])

In [230]:
print(googl_ma_r2_score)

-0.0003990186302342824


## Predicitons on validation data

#### IBM stock

In [231]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [232]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [233]:
#print(ibm_ma_mse_loss)

In [234]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [235]:
ibm_ma_mae_loss = mean_absolute_error(df_ibm_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[0])

In [236]:
print(ibm_ma_mae_loss)

15.625399017333985


Root Mean Squared Error

In [237]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[0], squared=False)

In [238]:
print(ibm_ma_mse_loss)

16.019844749317965


Mean Absolute Percentage Error

In [239]:
ibm_ma_map_loss = mean_absolute_percentage_error(df_ibm_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[0])

In [240]:
print(ibm_ma_map_loss)

0.11439338522126555


R<sup>2</sup> score

In [241]:
ibm_ma_r2_score = r2_score(df_ibm_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[0])

In [242]:
print(ibm_ma_r2_score)

-19.559895325172956


In [243]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [244]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [245]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [246]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [247]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [248]:
#print(aapl_ma_mse_loss)

In [249]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [250]:
aapl_ma_mae_loss = mean_absolute_error(df_aapl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[1])

In [251]:
print(aapl_ma_mae_loss)

3.9836158370971675


##### Root Mean Squared Error

In [252]:
aapl_ma_rmse_loss = mean_squared_error(df_aapl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[1], squared=False)

In [253]:
print(aapl_ma_rmse_loss)

5.171675080450402


##### Mean Absolute Percentage Error

In [254]:
aapl_ma_map_loss = mean_absolute_percentage_error(df_aapl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[1])

In [255]:
print(aapl_ma_map_loss)

0.03157532987150507


R<sup>2</sup> score

In [256]:
aapl_ma_r2_score = r2_score(df_aapl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[1])

In [257]:
print(aapl_ma_r2_score)

-0.10065255569739517


In [258]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [259]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [260]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [261]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [262]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [263]:
#print(amzn_ma_mse_loss)

In [264]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [265]:
amzn_ma_mae_loss = mean_absolute_error(df_amzn_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[2])

In [266]:
print(amzn_ma_mae_loss)

102.77173828124997


Root Mean Squared Error

In [267]:
amzn_ma_rmse_loss = mean_squared_error(df_amzn_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[2], squared=False)

In [268]:
print(amzn_ma_rmse_loss)

115.0337363178299


##### Mean Absolute Percentage Error

In [269]:
amzn_ma_map_loss = mean_absolute_percentage_error(df_amzn_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[2])

In [270]:
print(amzn_ma_map_loss)

0.03142836701414785


R<sup>2</sup> score

In [271]:
amzn_ma_r2_score = r2_score(df_amzn_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[2])

In [272]:
print(amzn_ma_r2_score)

-0.07217396529979303


In [273]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [274]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [275]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [276]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [277]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [278]:
#print(googl_ma_mse_loss)

In [279]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [280]:
googl_ma_mae_loss = mean_absolute_error(df_googl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[3])

In [281]:
print(googl_ma_mae_loss)

178.49924560546873


Root Mean Squared Error

In [282]:
googl_ma_rmse_loss = mean_squared_error(df_googl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[3], squared=False)

In [283]:
print(googl_ma_rmse_loss)

196.5665507111728


##### Mean Absolute Percentage Error

In [284]:
googl_ma_map_loss = mean_absolute_percentage_error(df_googl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[3])

In [285]:
print(googl_ma_map_loss)

0.07889885121042675


R<sup>2</sup> score

In [286]:
googl_ma_r2_score = r2_score(df_googl_valid.iloc[:]['Adj Close'], ser_valid_ma_bmk[3])

In [287]:
print(googl_ma_r2_score)

-4.676619945601237


In [288]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [289]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [290]:
#print(googl_ma_mse_loss)

Volatility

In [291]:
print(volatility(df_ibm_valid['Adj Close'], n))

0.6569648074393963


In [292]:
print(volatility(df_aapl_valid['Adj Close'], n))

1.2789648716406885


In [293]:
print(volatility(df_amzn_valid['Adj Close'], n))

649.5784275936765


In [294]:
print(volatility(df_googl_valid['Adj Close'], n))

358.2414334100405


As expected, loss augments as we observe moving average on larger windows.
Also, we can observe that Amazon.com and Alphabet Inc. have greater losses, that also corresponds to higher volatility.